In [1]:
import codecs
import csv
import time
import os
import re
import gzip
import pandas as pd
import numpy as np

In [2]:
PATH_TO_DATA = 'Data/'
PATH_TO_DATA_EN = PATH_TO_DATA+"enwiki/"
PATH_TO_DATA_UK = PATH_TO_DATA+"ukwiki/"

In [3]:
ENWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA_EN):
    if re.match(r"enwiki-20180620-pages-meta-current\d{2}-p\d+p\d+.xml_art.csv.gz", file):
        ENWIKI_ART_FNMS.append(file)
ENWIKI_RED_FNMS = []
for file in os.listdir(PATH_TO_DATA_EN):
    if re.match(r"enwiki-20180620-pages-meta-current\d{2}-p\d+p\d+.xml_red.csv.gz", file):
        ENWIKI_RED_FNMS.append(file)        

In [4]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [5]:
def pack_and_remove(file_name):
    file_name_new = file_name+'.gz'
    with open(file_name, 'rb') as f_in, gzip.open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(file_name)
    return file_name_new

In [50]:
df_title_count = None
for fn in ENWIKI_ART_FNMS:
    fn = PATH_TO_DATA_EN+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='ISO-8859-1',quotechar="'")

    df_red_links = df_articles[df_articles['is_red_link'] == True]
    df_title_count_tmp = pd.DataFrame(df_red_links.groupby('link_val').link_val.count())
    df_title_count_tmp.columns = ['link']
    df_title_count_tmp = df_title_count_tmp.reset_index(col_level=1)
    df_title_count_tmp.columns = ['link_title', 'in_count']
    
    if df_title_count is not None:
        df_title_count = df_title_count.append(df_title_count_tmp)        
        #print("append")
    else:
        df_title_count = df_title_count_tmp
        #print("assign")    
    df_title_count = df_title_count.groupby('link_title').in_count.sum().copy()
    df_title_count.columns = ['link']
    df_title_count = df_title_count.reset_index()
    df_title_count.columns = ['link_title', 'in_count']        
    print("df_title_count size: {}".format(df_title_count.shape))
    os.remove(fn_new)

Data/enwiki/enwiki-20180620-pages-meta-current01-p10p30303.xml_art.csv.gz
df_title_count size: (282312, 2)
Data/enwiki/enwiki-20180620-pages-meta-current02-p30304p88444.xml_art.csv.gz
df_title_count size: (534722, 2)
Data/enwiki/enwiki-20180620-pages-meta-current03-p88445p200507.xml_art.csv.gz
df_title_count size: (804663, 2)
Data/enwiki/enwiki-20180620-pages-meta-current04-p200511p352689.xml_art.csv.gz
df_title_count size: (1124147, 2)
Data/enwiki/enwiki-20180620-pages-meta-current05-p352690p565313.xml_art.csv.gz
df_title_count size: (1448244, 2)
Data/enwiki/enwiki-20180620-pages-meta-current06-p565314p892912.xml_art.csv.gz
df_title_count size: (1757356, 2)
Data/enwiki/enwiki-20180620-pages-meta-current07-p892914p1268691.xml_art.csv.gz
df_title_count size: (2063623, 2)
Data/enwiki/enwiki-20180620-pages-meta-current08-p1268692p1791079.xml_art.csv.gz
df_title_count size: (2386929, 2)
Data/enwiki/enwiki-20180620-pages-meta-current09-p1791080p2336422.xml_art.csv.gz
df_title_count size: (2

In [66]:
df_title_count.to_csv(PATH_TO_DATA_EN+'enwiki-20180620-red_name_count.csv', index = False, encoding='ISO-8859-1', quotechar="'", escapechar ="\\")

In [59]:
df_title_count = df_title_count.sort_values(['in_count'], ascending=[0])

In [60]:
df_title_count

,link_title,in_count
756375,Allmusic,79292
2101603,Countries of the world,68096
4932857,List of countries,63259
7047750,Rural Districts of Iran,63203
3928707,Iran Daylight Time,59854
6734046,Race (United States Census),58224
8964302,association football,54693
6990032,Romanize,54656
9391627,population density,46853
5424485,Member of Parliament,42947


In [65]:
df_title_count.describe()

,in_count
count,9.655165e+06
mean,4.875887e+00
std,1.005153e+02
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,2.000000e+00
max,7.929200e+04


In [54]:
df_count_by_count = pd.DataFrame(df_title_count.groupby('in_count').in_count.count())

In [56]:
df_count_by_count.columns = ['link']
df_count_by_count = df_count_by_count.reset_index()
df_count_by_count.columns = ['count', 'in_count']  

In [70]:
df_count_by_count = df_count_by_count.sort_values(['count'], ascending=[1])

In [71]:
df_count_by_count

,count,in_count
0,1,6133435
1,2,1448269
2,3,600666
3,4,336680
4,5,208796
5,6,145639
6,7,106617
7,8,80978
8,9,63738
9,10,51641


In [64]:
df_count_by_count.describe()

,count,in_count
count,2724.000000,2.724000e+03
mean,2624.078561,3.544481e+03
std,5130.918459,1.215843e+05
min,1.000000,1.000000e+00
25%,681.750000,1.000000e+00
50%,1393.500000,2.000000e+00
75%,2618.250000,1.000000e+01
max,79292.000000,6.133435e+06


In [67]:
df_count_by_count.to_csv(PATH_TO_DATA_EN+'enwiki-20180620-red_count_by_count.csv', index = False, encoding='ISO-8859-1', quotechar="'", escapechar ="\\")